In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 623f8c4b-ab80-4042-bd09-0e14075ebb37, 4, Finished, Available, Finished)

**Description

In this step, I moved raw CSV files from the Landing Zone into the Bronze Layer. We applied a fixed schema, standardized column names, and saved the data into Delta tables. This ensures that raw data is stored in a structured, query-ready format without altering its content.

Key Points

Defined schemas for Sales and Rebate files so Spark reads them consistently.

Loaded raw CSV files from the Landing Zone into Spark DataFrames.

Normalized column names (lowercase, underscores, no spaces).

Saved data into Delta tables (bronze_sales_raw, bronze_rebates_raw).

Used overwrite mode to always refresh Bronze with the latest landing data.**

In [3]:
from pyspark.sql.types import *
from pyspark.sql import functions as F

# Paths to landing
sales_path = "Files/landing/sales_raw"
rebates_path = "Files/landing/rebates_raw"

# Define schemas (align with your CSVs)
sales_schema = StructType([
    StructField("Sale Transaction ID", StringType(), True),
    StructField("Sales Rep ID", StringType(), True),
    StructField("Sales Rep Name", StringType(), True),
    StructField("Sales Rep Country", StringType(), True),
    StructField("Sales Rep Country Code", StringType(), True),
    StructField("Sales Rep Region", StringType(), True),
    StructField("Sales Country Code", StringType(), True),
    StructField("Customer ID", StringType(), True),
    StructField("Customer Name", StringType(), True),
    StructField("Customer Country", StringType(), True),
    StructField("Customer Region", StringType(), True),
    StructField("Customer Country Code", StringType(), True),
    StructField("Unit Quantity", StringType(), True),
    StructField("Sale Amount", StringType(), True),
    StructField("Sale Datetime", StringType(), True),
    StructField("Product Details", StringType(), True),
    StructField("Product Code", StringType(), True),
    StructField("Sale Location", StringType(), True),
    StructField("CurrencyCode", StringType(), True),
])

rebate_schema = StructType([
    StructField("Sales Rep ID", StringType(), True),
    StructField("Sales Rep Name", StringType(), True),
    StructField("Sales Rep Region", StringType(), True),
    StructField("Customer Country", StringType(), True),
    StructField("Customer Region", StringType(), True),
    StructField("Sale Location", StringType(), True),
    StructField("CurrencyCode", StringType(), True),
    StructField("Customer Name", StringType(), True),
    StructField("Sale Transaction ID", StringType(), True),
    StructField("Customer ID", StringType(), True),
    StructField("Rebate Datetime", StringType(), True),
    StructField("Rebate Transaction ID", StringType(), True),
    StructField("Rebate Type", StringType(), True),
    StructField("Rebate Amount", StringType(), True),
])

# Read landing files
df_sales = spark.read.option("header", True).schema(sales_schema).csv(sales_path)
df_rebates = spark.read.option("header", True).schema(rebate_schema).csv(rebates_path)

# Normalize column names (snake_case)
def normalize_cols(df):
    for c in df.columns:
        df = df.withColumnRenamed(c, c.strip().lower().replace(" ", "_"))
    return df

df_sales = normalize_cols(df_sales)
df_rebates = normalize_cols(df_rebates)

# Truncate-load Bronze (overwrite every run)
df_sales.write.format("delta").mode("overwrite").saveAsTable("bronze_sales_raw")
df_rebates.write.format("delta").mode("overwrite").saveAsTable("bronze_rebates_raw")


StatementMeta(, 623f8c4b-ab80-4042-bd09-0e14075ebb37, 5, Finished, Available, Finished)

In [4]:
%%sql
SELECT * 
FROM bronze_sales_raw
LIMIT 10;


StatementMeta(, 623f8c4b-ab80-4042-bd09-0e14075ebb37, 6, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 19 fields>

In [5]:
spark.table("bronze_sales_raw").printSchema()
spark.table("bronze_sales_raw").show(5, truncate=False)


StatementMeta(, 623f8c4b-ab80-4042-bd09-0e14075ebb37, 7, Finished, Available, Finished)

root
 |-- sale_transaction_id: string (nullable = true)
 |-- sales_rep_id: string (nullable = true)
 |-- sales_rep_name: string (nullable = true)
 |-- sales_rep_country: string (nullable = true)
 |-- sales_rep_country_code: string (nullable = true)
 |-- sales_rep_region: string (nullable = true)
 |-- sales_country_code: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_country: string (nullable = true)
 |-- customer_region: string (nullable = true)
 |-- customer_country_code: string (nullable = true)
 |-- unit_quantity: string (nullable = true)
 |-- sale_amount: string (nullable = true)
 |-- sale_datetime: string (nullable = true)
 |-- product_details: string (nullable = true)
 |-- product_code: string (nullable = true)
 |-- sale_location: string (nullable = true)
 |-- currencycode: string (nullable = true)

+-------------------+------------+----------------+-----------------+----------------------+-----------